# Mixed models

Here, we try fitting generalized linear mixed effect models (GLMMs), i.e. models of the form:

\begin{align}
\mathbb{E}\left[ y_{ij} \mid \boldsymbol \gamma_i \right] &= h \left( \boldsymbol \eta_{ij} \right) \\
\boldsymbol \eta_{ij} & = \mathbf{x}_{ij}^T\boldsymbol \beta + \mathbf{u}_{ij}^T \boldsymbol \gamma_i
\end{align}

for data $(y_{ij}, \mathbf{x}_{ij})$ that is grouped into $i \in \{1, \dots, m \}$ groups (or clusters) and $j \in \{1, \dots, n_i\}$ observations per group.

For an intro on the notation, usual assumptions, etc. please refer to _any_ book on regression models, as this is not part of this notebook. The most prominrnt example of an LMM is for Gaussian responses $\mathbf{y}$:

\begin{align}
y_{ij} \mid \boldsymbol \gamma_i & \sim \mathcal{N}\left(\mathbf{x}_{ij}^T\boldsymbol \beta + \mathbf{u}_{ij}^T\boldsymbol \gamma_i, \sigma^2\right)\\
\boldsymbol \gamma_i & \sim  \mathcal{N}\left(\mathbf{0}, \mathbf{Q}\right)
\end{align}

where the response function $h$ is the identity function. Another frequent one in the exponential family is the Poisson case:

\begin{align}
{y}_{ij} \mid \boldsymbol \gamma_i & \sim \text{Pois} \left( \exp \left( \mathbf{x}_{ij}^T\boldsymbol \beta + \mathbf{u}_{ij}^T\boldsymbol \gamma_i \right)\right)\\
\boldsymbol \gamma_i & \sim  \mathcal{N}\left(\mathbf{0}, \mathbf{Q}\right)
\end{align}

where $h$ is for instance the exponential function. The main difference to classic linear regression models is that we also need to estimate the variance of the random effects $\boldsymbol \gamma$ which makes the model quite difficult to fit albeit its simplistic form. In the following we show, how $\boldsymbol \beta$ can be estimated and $\boldsymbol \gamma$ can be predicted for the Gaussian case and for the Poisson case. The implementations are not really efficient and don't use results from contemporary research (i.e., `lme4`'s PLS and PIRLS).

**I do not take warranty for the correctness or completeness of this document.**

The relevant code can be found [here](https://github.com/dirmeier/mixed-models).

We first load some libraries we need.

In [35]:
import scipy as sp
import pandas
import pandas as pd
from patsy import dmatrices, dmatrix
import scipy.stats as st
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

rmvnorm = st.multivariate_normal.rvs
rpois = st.poisson.rvs

The implementations for fitting GLMMs can be found in these modules:

In [6]:
from lme.optim import optim, optim_poisson
from lme.ls import wls, solve_gamma, working_response, working_weight, irls
from lme.marginal_likelhood import restricted_mll, profile_mll
from lme.util import block_diag, cholesky_factor, v, ranef_variance

We use the design matrix from the sleep study data from `lme4` since it's good for our purpose (but any other one will do, too). The description of the data from the package: *The average reaction time per day for subjects in a sleep deprivation study. On day 0 the subjects had their normal amount of sleep. Starting that night they were restricted to 3 hours of sleep per night. The observations represent the average reaction time on a series of tests given each day to each subject.*

In [49]:
tab = pandas.read_csv("./data/sleepstudy.csv")
tab.head()

,Reaction,Days,Subject
0,249.5600,0,308
1,258.7047,1,308
2,250.8006,2,308
3,321.4398,3,308
4,356.8519,4,308


In `R`'s formula notation we are trying to fit a model of this form: `Reaction ~ Days + (Days | Subject)`. At the time of writing this `patsy` doesn't support creating random effects model matrices, so we need to do it ourselves. However, we can use it for the response matrix and fixed effects design matrix.

In [67]:
y, X = dmatrices("Reaction ~ Days", tab)

To create the random effects matrix $\mathbf{U}$, let us first realize how $\mathbf{U}$ has to look: if we collect all cluster-specific responses $\mathbf{y}_i$ and the respective random effects $\boldsymbol \gamma_i$ into vectors, we get

\begin{align}
\mathbf{y} = 
\begin{pmatrix}
\mathbf{y}_1 \\
\vdots\\
\mathbf{y}_m\\
\end{pmatrix} =
\begin{pmatrix}
y_{1,1}\\
\vdots\\
y_{1,n_1}\\
y_{2,1}\\
\vdots\\
y_{m,n_m}
\end{pmatrix}
,\qquad
\boldsymbol \gamma = 
\begin{pmatrix}
\boldsymbol \gamma_{1}\\
\vdots\\
\boldsymbol \gamma_{m}
\end{pmatrix}
\end{align}

Thus, in matrix notation the general form of $\mathbf{U}$ needs to be a block diagonal matrix:

\begin{align}
\mathbf{U} = \text{blockdiag}\left(\mathbf{U}_1, \dots, \mathbf{U}_i, \dots ,\mathbf{U}_m\right)
\end{align}


We can compute $\mathbf{U}$ using the following method:

In [65]:
def build_ranef_model_matrix(tab, factor, ranef):
    inter_tab = tab[[factor, ranef]].copy()
    inter_tab['grp'] = LabelEncoder().fit_transform(tab.Subject)
    inter_tab = inter_tab[[factor, "grp"]].pivot(columns=factor).reindex()
    inter_tab.values[sp.isfinite(inter_tab.values)] = 1
    inter_tab.values[sp.isnan(inter_tab.values)] = 0

    slope_tab = tab[[factor, ranef]].copy().pivot(columns=factor).reindex()
    slope_tab.values[sp.isnan(slope_tab.values)] = 0

    U = pd.concat([inter_tab, slope_tab], axis=1, sort=True)
    U = U.reindex(sorted(U.columns, key=lambda x: x[1]), axis=1)

    return sp.asarray(U.values)

In [66]:
U = build_ranef_model_matrix(tab, "Subject", "Days")
U

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 2., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 7.],
       [0., 0., 0., ..., 0., 1., 8.],
       [0., 0., 0., ..., 0., 1., 9.]])

Let's do some checks: $\mathbf{U}$ needs to have as many rows as $\mathbf{X}$ or $\mathbf{y}$ and twice as many columns as grouping factors, i.e. an intercept and a slope for every group.

In [80]:
assert(U.shape[0] == X.shape[0])
assert(U.shape[1] == 2 * len(sp.unique(tab.Subject.values)))

As a first example, we sample the responses ourselves to be able to validate the inferences. We set $\sigma^2 = 0.1$ and $\boldsymbol \beta = \left(2 \ 1\right)^T$.

In [82]:
n, p = X.shape
q = int(U.shape[1] / 2)

sd = 0.1
beta = sp.array([2, 1])

Then we sample every random effects vector $\boldsymbol \gamma_i \sim \mathcal{N}\left(\mathbf{0}, \sigma^2\begin{pmatrix} 1 & 0.25 \\ 0.25 & 1\end{pmatrix}\right)$. Like $\mathbf{U}$, the covariance matrix of $\boldsymbol \gamma$, $\mathbf{G}$,  has to be block-diagonal, too:


\begin{align}
\mathbf{G} = \text{blockdiag}\left(\mathbf{Q}, \dots, \mathbf{Q}, \dots ,\mathbf{Q}\right)
\end{align}


In [93]:
sp.random.seed(23)

Q = sd * sp.array([[1, 0.25], [0.25, 1]])
gamma = rmvnorm(mean=sp.zeros(q * 2), cov=block_diag(Q, q))
gamma.mean()

0.004416309719067267

We can then sample from the conditional distribution $P(\mathbf{y} \mid \boldsymbol \gamma)$ as:

In [88]:
y = rmvnorm(mean=X.dot(beta) + U.dot(gamma), cov=sp.diag(sd * sp.ones(n)))

Fitting an LMM consists of estimating $\sigma^2$, $\boldsymbol \beta$ and $\mathbf{Q}$ and predicting the random effects $\boldsymbol \gamma$. First we estimate $\sigma^2$ and $\mathbf{Q}$ by integrating out $\boldsymbol \beta$ and $\boldsymbol \gamma$ from the conditional likelihood. To do so we treat $\boldsymbol \beta$ as a random variable with flat prior as from an empirical Bayes perspective. Having estimated the variance components, we estimate $\boldsymbol \beta$ and $\boldsymbol \gamma$. 

More specifically, we start from the joint likelihood of all parameters $L\left(\boldsymbol \beta, \boldsymbol \gamma, \sigma^2, \mathbf{G} \right)$. From this we marginalize out the random effects 

\begin{align}
L\left(\boldsymbol \beta, \sigma^2, \mathbf{G} \right) = \int L\left(\boldsymbol \beta, \boldsymbol \gamma, \sigma^2, \mathbf{G} \right) d\boldsymbol \gamma
\end{align}

As mentioned above, we now treat $\boldsymbol \beta$ as random and marginalize it out, too:

\begin{align}
L\left(\sigma^2, \mathbf{G} \right) = \int L\left(\boldsymbol \beta, \sigma^2, \mathbf{G} \right) d\boldsymbol \beta
\end{align}

We usually call the likelihood above *restricted* likelihood. $L\left(\sigma^2, \mathbf{G} \right)$ needs to be estimated numerically, for instance using Newton's method. We then use [Henderson's mixed model equations](https://en.wikipedia.org/wiki/Mixed_model#Estimation) for $\boldsymbol \beta$ and $\boldsymbol \gamma$. 

We optimize the (log)likelihood function first:

In [95]:
pll = restricted_mll("gaussian")
fn = lambda nu, y, X, U: sp.asscalar(pll(nu, y, X, U)[0])
optimz = optim(fn, y, X, U)
sd_hat, nu_hat = optimz['sigma'], optimz['nu']

Estimates of $\sigma^2$:

In [119]:
print("sigma:\t\t{}\nsigma_hat:\t{}".format(sd, sd_hat))

sigma:		0.1
sigma_hat:	0.09291316710177702


Estimates of $\mathbf{G}$:

In [124]:
print("Q:\n{}\nQ_hat:\n{}".format(Q, cholesky_factor(nu_hat)))

Q:
[[0.1   0.025]
 [0.025 0.1  ]]
Q_hat:
[[0.05602312 0.02232002]
 [0.02232002 0.12631985]]


We then use Henderson's equations (see McCulloch & Searle (2001), Eqns 6.24 and 6.42):

In [127]:
V_hat, G_hat, R_hat = v(sd_hat, nu_hat, n, q, U)
b_hat = wls(y, X, V_hat)
gamma_hat = solve_gamma(y, X, G_hat, U, V_hat, b_hat)

Estimates of $\boldsymbol \beta$:

In [133]:
print("beta: {}\nbeta_hat: {}".format(beta, b_hat))

beta: [2 1]
beta_hat: [2.01487263 0.909761  ]


Estimates of $\boldsymbol \gamma$:

In [135]:
print("gamma:\n{}\ngamma_hat:\n{}".format(gamma, gamma_hat))

gamma:
[-0.25538236 -0.07811166 -0.16371937  0.15081283  0.37619662  0.01261309
 -0.48256002  0.0082431  -0.04384917 -0.30698673  0.06033511  0.46520567
  0.27633971 -0.09256565  0.52124719  0.04748265 -0.05064887  0.71172263
 -0.08712428 -0.79900486  0.125951    0.0477785  -0.27174966 -0.06332042
 -0.12725791 -0.03387785  0.01130221 -0.04147368  0.22723849  0.29448651
 -0.02159007  0.526561   -0.25158029  0.03071211 -0.17614052 -0.3882979 ]
gamma_hat:
[ 0.1948704   0.64089984 -0.06781785  0.50560475  0.36041629  0.37021943
 -0.13094845 -0.01177385  0.16812377 -0.5483902   0.00250318  0.00421239
  0.07587806  0.00726997  0.08171275  0.29203514 -0.01488344 -0.28991013
  0.02992848  0.11913799  0.05762302 -0.31275395 -0.21977802 -0.18829479
  0.09773335 -0.17207727 -0.07876445 -0.10774269  0.11157798 -0.2421992
 -0.01439661  0.03826879 -0.1055833   0.49814128 -0.54819518 -0.6026475 ]


Let's our implementation to `statsmodels`.

In [141]:
import statsmodels.formula.api as smf

In [142]:
tab = pandas.read_csv("./data/sleepstudy.csv")
tab.head()

,Reaction,Days,Subject
0,249.5600,0,308
1,258.7047,1,308
2,250.8006,2,308
3,321.4398,3,308
4,356.8519,4,308


In [147]:
y, X = dmatrices("Reaction ~ Days", tab)

In [154]:
md = smf.mixedlm("Reaction ~ Days", tab, groups=tab["Subject"], re_formula="~ Days")
mdf = md.fit()
print(mdf.summary())

            Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   Reaction 
No. Observations:    180       Method:               REML     
No. Groups:          18        Scale:                654.9405 
Min. group size:     10        Likelihood:           -871.8141
Max. group size:     10        Converged:            Yes      
Mean group size:     10.0                                     
--------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------
Intercept        251.405    6.825 36.838 0.000 238.029 264.781
Days              10.467    1.546  6.771 0.000   7.438  13.497
Group Var        612.096   11.881                             
Group x Days Cov   9.605    1.821                             
Days Var          35.072    0.610                             



In [196]:
optimz = optim(fn, y, X, U)
sd_hat, nu_hat = optimz['sigma'], optimz['nu']
V_hat, G_hat, R_hat = v(sd_hat, nu_hat, n, q, U)
b_hat = wls(y, X, V_hat)
gamma_hat = solve_gamma(y, X, G_hat, U, V_hat, b_hat)

In [197]:
b_hat

array([[251.40510485],
       [ 10.46728596]])

## Poisson GLMMs

In [13]:
def glme():
    n, pa = X.shape
    q = int(U.shape[1] / 2)

    sd = 0.1
    beta = sp.array([.1, .1])
    Q = sd * sp.array([[1, 0.25], [0.25, 1]])
    gamma 
    
    
    
    
    = rmvnorm(mean=sp.zeros(q * 2), cov=block_diag(Q, q))

    y = rpois(mu=sp.exp(X.dot(beta) + U.dot(gamma)))

    pll = restricted_mll("poisson")
    fn = lambda nu, y, X, U, W, b: sp.asscalar(pll(nu, y, X, U, W, b)[0])

    b_tilde = bold = sp.ones(shape=p)
    g_tilde = gold = sp.ones(shape=q * 2)
    
    while True:
        y_tilde = working_response(y, X, U, b_tilde, g_tilde, sp.exp, sp.exp)
        w_tilde = working_weight(y, X, U, b_tilde, g_tilde, sp.exp, sp.exp)
        
        optimz = optim_poisson(fn, y_tilde, X, U,
                               sp.diag(w_tilde), b_tilde, iter=1)
        nu_hat = optimz['nu']
        
        G_hat = ranef_variance(nu_hat, q)
        b_tilde, g_tilde = irls(X, U, G_hat, w_tilde, y_tilde)

        if sp.sum((b_tilde - bold) ** 2) < 0.00001 and \
           sp.sum((g_tilde - gold) ** 2) < 0.00001:
           break
        bold, gold = b_tilde, g_tilde
    
    print("beta/beta_hat:\n{}/{}\n".format(beta, b_tilde))
    print("gamma/gammahat:\n{}/\n{}\n".format(gamma, g_tilde))

In [14]:
glme()

beta/beta_hat:
[0.1 0.1]/[-0.19744485 -0.04434282]

gamma/gammahat:
[ 0.14940994 -0.35028678 -0.63948735  0.07552716  0.46248431 -0.01636515
  0.09370921 -0.38934835  0.27404765  0.44083184 -0.15531892  0.0752115
  0.0396481  -0.0703771   0.23669006 -0.04031613  0.04507746  0.2424409
 -0.51458518 -0.3319769   0.07864307 -0.03069841  0.42996693 -0.04190384
 -0.13766715 -0.15954766  0.18798917 -0.06912578 -0.15647169 -0.35374621
 -0.40461758 -0.24930647 -0.0647015   0.12580602 -0.24023007 -0.13100664]/
[-0.47211929  0.05051085 -0.28029166  0.27171182  1.10779135  0.06555264
 -0.35421056 -0.6444423   0.97925192  0.52865871 -0.09118659  0.27474221
 -0.3742151   0.14417273 -0.54953846  0.17835032  0.68189258  0.34449695
 -0.11954577 -1.29927973  0.1181167   0.15460799 -0.11423731  0.1978962
 -0.16812165 -0.08938988  0.8840803  -0.15482671 -0.40312473 -0.09066562
 -0.20111074 -0.3320241  -0.27299537  0.33566633 -0.37043563  0.06426158]

